### Caching and Time Comparison:

For Caching and Time Comparison, we will be using the same dataset as before one. We will first run the heavy query, count the total rows without caching first and calculate the time consumed for executing it. Then, we will cache the heavy query df, count the total rows and calculate the total time consumed. This way, we can understand the significance of caching

In [0]:
df = spark.read.table("dai.default.sales_transactions")
display(df.head(10))

transactionID,customerID,franchiseID,dateTime,product,quantity,unitPrice,totalPrice,paymentMethod,cardNumber
1002961,2000253,3000047,2024-05-14T12:17:01.495Z,Golden Gate Ginger,8,3,24,amex,378154478982993
1003007,2000226,3000047,2024-05-10T23:10:10.239Z,Austin Almond Biscotti,36,3,108,mastercard,2244626981238094
1003017,2000108,3000047,2024-05-16T16:34:10.613Z,Austin Almond Biscotti,40,3,120,mastercard,2490570234487424
1003068,2000173,3000047,2024-05-02T04:31:51.612Z,Pearly Pies,28,3,84,amex,343808569426192
1003103,2000075,3000047,2024-05-04T23:44:26.902Z,Pearly Pies,28,3,84,visa,4377080942201798
1003147,2000295,3000047,2024-05-15T16:17:06.259Z,Austin Almond Biscotti,32,3,96,amex,371093774812677
1003196,2000237,3000047,2024-05-07T11:13:22.469Z,Tokyo Tidbits,40,3,120,mastercard,5538807345848392
1003329,2000272,3000047,2024-05-06T03:32:16.017Z,Outback Oatmeal,28,3,84,visa,4872480716880043
1001264,2000209,3000047,2024-05-16T17:32:28.547Z,Pearly Pies,28,3,84,mastercard,5287105980593305
1001287,2000120,3000047,2024-05-15T08:41:28.406Z,Austin Almond Biscotti,40,3,120,amex,376211012259783


In [0]:
import pyspark.sql.functions as F

df_heavy = df \
    .withColumn("price_double", F.col("totalPrice") * 2) \
    .withColumn("qty_squared", F.col("quantity") * F.col("quantity")) \
    .withColumn("price_bucket",
        F.when(F.col("totalPrice") > 500, "High")
         .when(F.col("totalPrice") > 200, "Medium")
         .otherwise("Low")
    ) \
    .filter(F.col("quantity") > 1)

df_heavy.createOrReplaceTempView("heavy_view")

In [0]:
import time

start = time.time()

spark.sql("""
SELECT franchiseID,
       SUM(qty_squared),
       AVG(price_double),
       COLLECT_SET(price_bucket)
FROM heavy_view
GROUP BY franchiseID
""").collect()

spark.sql("""
SELECT price_bucket,
       SUM(totalPrice),
       COUNT(*)
FROM heavy_view
GROUP BY price_bucket
""").collect()

end = time.time()

print("Time WITHOUT cache:", end - start)

Time WITHOUT cache: 0.9553961753845215


In [0]:
df_heavy.cache()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5351096978031476>, line 1
----> 1 df_heavy.cache()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:2126, in DataFrame.cache(self)
   2125 def cache(self) -> ParentDataFrame:
-> 2126     return self.persist()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:2133, in DataFrame.persist(self, storageLevel)
   2128 def persist(
   2129     self,
   2130     storageLevel: StorageLevel = (StorageLevel.MEMORY_AND_DISK_DESER),
   2131 ) -> ParentDataFrame:
   2132     relation = self._plan.plan(self._session.client)
-> 2133     self._session.client._analyze(
   2134         method="persist", relation=relation, storage_level=storageLevel
   2135     )
   2136     return self

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/co

In [0]:
df_heavy.persist()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5351096978031477>, line 1
----> 1 df_heavy.persist()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:2133, in DataFrame.persist(self, storageLevel)
   2128 def persist(
   2129     self,
   2130     storageLevel: StorageLevel = (StorageLevel.MEMORY_AND_DISK_DESER),
   2131 ) -> ParentDataFrame:
   2132     relation = self._plan.plan(self._session.client)
-> 2133     self._session.client._analyze(
   2134         method="persist", relation=relation, storage_level=storageLevel
   2135     )
   2136     return self

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1859, in SparkConnectClient._analyze(self, method, **kwargs)
   1857     raise SparkConnectException("Invalid state during retry exception handling.")
   1858 except Exception as error:
->

In [0]:
spark.sql("CACHE TABLE heavy_view")
spark.sql("SELECT COUNT(*) FROM heavy_view").collect()   # materialize

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5351096978031473>, line 1
----> 1 spark.sql("CACHE TABLE heavy_view")
      2 spark.sql("SELECT COUNT(*) FROM heavy_view").collect()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/session.py:879, in SparkSession.sql(self, sqlQuery, args, **kwargs)
    876         _views.append(SubqueryAlias(df._plan, name))
    878 cmd = SQL(sqlQuery, _args, _named_args, _views)
--> 879 data, properties, ei = self.client.execute_command(cmd.command(self._client))
    880 if "sql_command_result" in properties:
    881     df = DataFrame(CachedRelation(properties["sql_command_result"]), self)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1589, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
   1587     req.user_context.user_id = s

---
## When we use:
`df.cache()`
Spark stores the DataFrame in memory after the first action.

So:
- First action → compute + store in memory
- Subsequent actions → reuse in-memory data
- Avoids recomputation of transformations
- Improves performance in iterative workloads

Caching is especially useful when:
- The same DataFrame is reused multiple times
- Performing multiple aggregations
- Running ML pipelines

---

# Serverless Limitation

In Databricks Serverless Community Edition, memory-level persistence (`cache()`, `persist()`, `CACHE TABLE`) is restricted.

So while the performance difference could not be demonstrated practically, conceptually:
> Caching reduces lineage recomputation and improves execution efficiency in distributed Spark workloads.